# Running the ablation gradient model on MODICE clean ice hypsometry

## <i><p>Purpose of this notebook:  Introduce you to:     <ol>         <li> a few ways to make a plot nicer looking</li>         <li> an implementation of the ablation-gradient model, using MODICE hypsometry for clean ice</li>     </ol> </p>     <p>At the end of this lesson, you should be able to read a MODICE hypsometry file, and:     <ol>         <li> run the clean ice ablation gradient model for several basins and different inputs, and         <li> compare results     </ol> </p></i>

In [ ]:
%pylab notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from imp import reload
pd.set_option('display.max_rows', 370)
pd.set_option('display.max_columns', 90)
pd.set_option('display.width', 200)

## Read a MODICE file as a python hypsometry object:

In [ ]:
import hypsometry
reload(hypsometry)
filename = "modice_area_by_elev/IN_Indus_at_Kotri.0100m.area_by_elev.txt"
modice = hypsometry.Hypsometry()
modice.read( filename, verbose=True )

In [ ]:
modice.data.columns = [float(col) for col in modice.data.columns]
modice.data

## Display it as a bar plot (change the default y axis labelling)

In [ ]:
modice.data.ix[0].plot( title='IN_Indus_at_Kotri MODICE by elevation', kind='barh',figsize=(12,8) )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.set_title('IN_Indus_at_Kotri MODICE by elevation')
ax.set_xlabel('MODICE Area($km^2$)')
ax.set_ylabel('Elevation ($m$)')
modice.data.ix[0].plot( kind='barh',figsize=(12,12))
ax.set_yticks( ax.get_yticks()[::10] )
ax.set_yticklabels( modice.data.columns[::10] )

## Use this MODICE area data to run the ablation model for clean ice, which is: <br> $Q = \sum\limits_{i=1}^n{b_{i} A_{i}}$ <br> for  <br>$Q$ = glacier ice melt volume, $m^3$ <br>$b_{i}$ = specific ice melt at elevation band $i$, $\frac{m}{100 m}$ <br>$A_{i}$ = clean ice area at elevation band $i$, $km^2$

Ref: Racoviteanu, A. E., R. Armstrong, and M. W. Williams (2013), Evaluation of an ice ablation model to estimate the contribution of melting glacier ice to annual discharge in the Nepal Himalaya, Water Resour. Res., 49, 5117–5133, doi:10.1002/wrcr.20370.

In [ ]:
import racovite_ablation_model
reload( racovite_ablation_model )
melt = racovite_ablation_model.run( modice, 5000., 0.6 )

In [ ]:
melt.data

In [ ]:
melt.data.max(axis=1)

In [ ]:
melt.comments

In [ ]:
melt.comments.append( "ELA=500m" )

In [ ]:
melt.comments.append( "db/dz=0.6m/100m" )

In [ ]:
melt.comments

In [ ]:
out_filename = "modice_area_by_elev/IN_Indus_at_Kotri.0100m.racovite_clean_ice_melt_by_elev.txt"
melt.write( out_filename, verbose=True )

In [ ]:
!cat modice_area_by_elev/IN_Indus_at_Kotri.0100m.racovite_clean_ice_melt_by_elev.txt

In [ ]:
melt.data.ix[0].plot( title='IN_Indus_at_Kotri ablation melt by elevation', kind='barh',figsize=(12,8) )

In [ ]:
subset = melt.data[melt.data.columns[20:60]].ix[0]
subset

In [ ]:
label = 'Total melt=%.2f $km^3$' % ( melt.data.sum( axis=1 ).values[ 0 ] )
label

In [ ]:
subset = melt.data[melt.data.columns[20:60]].ix[0]
fig, ax = plt.subplots(1,1)
ax.set_title('IN_Indus_at_Kotri ablation model melt from MODICE')
ax.set_xlabel('Melt ($km^3$)')
ax.set_ylabel('Elevation ($m$)')
#melt.data.ix[0].plot( title='IN_Indus_at_Kotri ablation melt by elevation', kind='barh',figsize=(12,12))
subset.plot( title='IN_Indus_at_Kotri ablation melt by elevation', kind='barh',figsize=(12,12), label=label)
ax.set_yticks( ax.get_yticks()[::10] )
ax.set_yticklabels( subset.index[::10] )
ax.legend(loc='best')

Exercise:  There are modice files for the 5 CHARIS basins in the directory called modice_area_by_elev.  Try running each basin with the following ELA and db/dz, plot the results, and compare them.
<table>
    <tr>
        <th>Basin Name</th>
        <th>ELA (m)</th>
        <th>db/dz (m/100m)</th>
    </tr>
    <tr>
        <td>AM_AmuDarya_at_Chatly</td>
        <td>4600</td>
        <td>?</td>
    </tr>
    <tr>
        <td>BR_Bramaputra_at_Bahadurabad</td>
        <td>5700</td>
        <td>0.6</td>
    </tr>
    <tr>
        <td>IN_Indus_at_Kotri</td>
        <td>5200</td>
        <td>0.7</td>
    </tr>
       <tr>
        <td>GA_Ganges_at_Paksey</td>
        <td>5700</td>
        <td>0.6</td>
    </tr>
    <tr>
        <td>SY_SyrDarya_at_TyumenAryk</td>
        <td>4100</td>
        <td>??</td>
    </tr>    
</table>
        

In [ ]:
basins = [ 'AM', 'SY', 'IN', 'GA', 'BR' ]
x1 = numpy.array( [ 1, 2, 3, 4, 5 ] )
x2 = x1 + 0.25
x3 = x2 + 0.25
area = numpy.array( [ 448967., 249068., 844673., 943244., 514184. ] )
ice = numpy.array( [ 12343.5, 2398.14, 29858.8, 10812.1, 18678.7 ] )
fig, ax = plt.subplots(1,1)
figsize( 12, 8 )
ax.set_title('CHARIS Basins')
ax.set_ylabel('Area (x $10^3$ $km^2$)')
plt.bar( x2, area / 1000., width=0.25, color='grey', label='Basin Area' )
plt.bar( x3, ice / 1000., width=0.25, color='blue', label='MODICE' )
ax.set_xticks( x3 )
ax.set_xticklabels( basins )
ax.legend(loc='best')
fig.savefig('charis_basin_ice_area.png', dpi=300 )  # change filename to .pdf to save to different format


In [ ]:
basins = [ 'AM', 'SY', 'IN', 'GA', 'BR' ]

x1 = numpy.array( [ 1, 2, 3, 4, 5 ] )
x2 = x1 + 0.25
x3 = x2 + 0.25
discharge = numpy.array( [ 43.2, 16.6, 90.5, 379.9, 690.5 ] )
discharge_stdev = numpy.array( [ 11.4, 8.1, 24.9, 77.4, 122.2 ] )
total = numpy.array( [ 100., 100., 100., 100., 100. ] )
high_melt = numpy.array( [ 9.2, 1.5, 42.6, 22.2, 88.5 ] )
mid_high_melt = numpy.array( [ 6.1, 1.0, 31.8, 15.6, 66.7 ] )
mid_low_melt = numpy.array( [ 5.2, 0.6, 23.8, 13.6, 63.2 ] )
low_melt = numpy.array( [ 3.4, 0.4, 17.7, 9.6, 47.6 ] )

#melt = numpy.array()
fig, ax = plt.subplots(1,1)
figsize( 12, 8 )
ax.set_title('Modeled Ice Melt (% of GRDC Discharge)', fontsize=20 )
ax.set_ylabel('% of Measured Discharge', fontsize=20 )
ax.set_ylim( [0,55] )
ax.set_xlim( [0,6] )
plt.plot( x1, high_melt / discharge * 100., linestyle='', markersize=20., marker='o', color='blue' )

plt.plot( x1, mid_high_melt / discharge * 100., linestyle='', markersize=20., marker='o', color='blue' )
plt.plot( x1, mid_low_melt / discharge * 100., linestyle='', markersize=20., marker='o', color='blue' )
plt.plot( x1, low_melt / discharge * 100., linestyle='', markersize=20., marker='o', color='blue' )
plt.text( 1, high_melt[0] / discharge[0] * 100. + 2., str(discharge[0])+' +/- '+str(discharge_stdev[0])+" $km^3$", horizontalalignment='center', fontsize=20 )
plt.text( 2, high_melt[1] / discharge[1] * 100. + 2., str(discharge[1])+' +/- '+str(discharge_stdev[1])+" $km^3$", horizontalalignment='center', fontsize=20 )
plt.text( 3, high_melt[2] / discharge[2] * 100. + 2., str(discharge[2])+' +/- '+str(discharge_stdev[2])+" $km^3$", horizontalalignment='center', fontsize=20 )
plt.text( 4, high_melt[3] / discharge[3] * 100. + 2., str(discharge[3])+' +/- '+str(discharge_stdev[3])+" $km^3$", horizontalalignment='center', fontsize=20 )
plt.text( 5, high_melt[4] / discharge[4] * 100. + 2., str(discharge[4])+' +/- '+str(discharge_stdev[4])+" $km^3$", horizontalalignment='center', fontsize=20 )
ax.set_xticks( x1 )
ax.set_xticklabels( basins, fontsize=20 )
ax.set_yticklabels( numpy.array([0,10,20,30,40,50]), fontsize=20 )
fig.savefig('charis_discharge_vs_ice_melt.ps', dpi=300 )  # change filename to .pdf to save to different format


Author: M. J. Brodzik, written for CHARIS workshop, Dehra Dun, India, 2014